In [1]:
import numpy
import scipy.optimize
import random
import string
from math import exp
from math import log

def parseData(fname):
  for l in open(fname):
    yield eval(l)

print ("Reading data...")
data = list(parseData("beer_50000.json"))
print ("done")

Reading data...
done


In [2]:
def feature(datum):
  feat = [1, datum['review/taste'], datum['review/appearance'], datum['review/aroma'], datum['review/palate'], datum['review/overall']]
  return feat

X = [feature(d) for d in data]
y = [d['beer/ABV'] >= 6.5 for d in data]

def inner(x,y):
  return sum([x[i]*y[i] for i in range(len(x))])

def sigmoid(x):
  return 1.0 / (1 + exp(-x))

In [3]:
##################################################
# Logistic regression by gradient ascent         #
##################################################

# NEGATIVE Log-likelihood
def f(theta, X, y, lam):
  loglikelihood = 0
  for i in range(len(X)):
    logit = inner(X[i], theta)
    loglikelihood -= log(1 + exp(-logit))
    if not y[i]:
      loglikelihood -= logit
  for k in range(len(theta)):
    loglikelihood -= lam * theta[k]*theta[k]
  # for debugging
  # print("ll =" + str(loglikelihood))
  return -loglikelihood

# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
  dl = [0]*len(theta)
  for i in range(len(X)):
    logit = inner(X[i], theta)
    for k in range(len(theta)):
      dl[k] += X[i][k] * (1 - sigmoid(logit))
      if not y[i]:
        dl[k] -= X[i][k]
  for k in range(len(theta)):
    dl[k] -= lam*2*theta[k]
  return numpy.array([-x for x in dl])

In [4]:
##################################################
# Train                                          #
##################################################

def train(lam, X_train, y_train):
  theta,_,_ = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X[0]), fprime, pgtol = 10, args = (X_train, y_train, lam))
  return theta

In [5]:
##################################################
# Predict                                        #
##################################################

def performance(theta, X_set, y_set):
  scores = [inner(theta,x) for x in X_set]
  predictions = [s > 0 for s in scores]
  correct = [(a==b) for (a,b) in zip(predictions,y_set)]
  acc = sum(correct) * 1.0 / len(correct)
  return acc

In [6]:
# Problem 1

# Splitting into thirds
one_third = int(len(X)/3)
two_third = 2*one_third

X_training_set = X[:one_third]
y_training_set = y[:one_third]

X_validation_set = X[one_third:two_third]
y_validation_set = y[one_third:two_third]

X_test_set = X[two_third:]
y_test_set = y[two_third:]

# define lambda
lam = 1.0

# train on training set
theta = train(lam, X_training_set, y_training_set)
print ("Training done!")

# get accuracy on validation set
acc = performance(theta, X_validation_set, y_validation_set)
print ("Accuracy on validation set: " + str(acc))

# get accuracy on test set
acc = performance(theta, X_test_set, y_test_set)
print ("Accuracy on test set: " + str(acc))

Training done!
Accuracy on validation set: 0.90027601104
Accuracy on test set: 0.577813774898


In [7]:
# Problem 2

# Define the words
words = ["lactic", "tart", "sour", "citric", "sweet", "acid", "hop", "fruit", "salt", "spicy"]

# Getting the word counts
def feature(datum):
  feat = [1]
  counts = [0]*len(words)
  mystring = datum['review/text']
  mystring = mystring.translate(str.maketrans('','',string.punctuation))
  mywords = mystring.split()
  for i in range (0, len(mywords)):
    for j in range (0, len(words)):
      if mywords[i].lower() == words[j]:
        counts[j] += 1
  feat = feat + counts
  return feat

X = [feature(d) for d in data]
y = [d['beer/ABV'] >= 6.5 for d in data]

# Splitting into thirds
one_third = int(len(X)/3)
two_third = 2*one_third

X_training_set = X[:one_third]
y_training_set = y[:one_third]

X_validation_set = X[one_third:two_third]
y_validation_set = y[one_third:two_third]

X_test_set = X[two_third:]
y_test_set = y[two_third:]

In [8]:
# Problem 3

def error_statistics(theta, X_set, y_set):
  acc = [0,0,0,0,0]
  scores = [inner(theta,x) for x in X_set]
  predictions = [s > 0 for s in scores]
  # True positives
  for (a,b) in zip(predictions,y_set):
    if a == True and b == True:
      acc[0] += 1
  # True negatives
  for (a,b) in zip(predictions,y_set):
    if a == False and b == False:
      acc[1] += 1
  # False positives
  for (a,b) in zip(predictions,y_set):
    if a == True and b == False:
      acc[2] += 1
  # False negatives
  for (a,b) in zip(predictions,y_set):
    if a == False and b == True:
      acc[3] += 1
  # Balanced Error Rate
  acc[4] = 1 - 0.5*(acc[0]/(acc[0]+acc[3]) + acc[1]/(acc[1]+acc[2]))
  return acc

# define lambda
lam = 1.0

# train on training set
theta = train(lam, X_training_set, y_training_set)
print ("Training done!")

# evaluate on test set
acc = error_statistics(theta, X_test_set, y_test_set)
print ("True positives in test set: " + str(acc[0]))
print ("True negatives in test set: " + str(acc[1]))
print ("False positives in test set: " + str(acc[2]))
print ("False negatives in test set: " + str(acc[3]))
print ("Balanced Error Rate on test set: " + str(acc[4]))

Training done!
True positives in test set: 5834
True negatives in test set: 206
False positives in test set: 10549
False negatives in test set: 79
Balanced Error Rate on test set: 0.49710325522021903


In [9]:
# Problem 4

# Redefine functions to better account for class imbalance

# A ratio of 1 means no imbalance
ratio = 1

# NEGATIVE Log-likelihood
def f(theta, X, y, lam):
  loglikelihood = 0
  for i in range(len(X)):
    logit = inner(X[i], theta)
    loglikelihood -= log(1 + exp(-logit))
    if not y[i]:
      loglikelihood -= logit
      loglikelihood -= (log(1 + exp(-logit)) + logit)*(ratio - 1)
  for k in range(len(theta)):
    loglikelihood -= lam * theta[k]*theta[k]
  # for debugging
  # print("ll =" + str(loglikelihood))
  return -loglikelihood

# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
  dl = [0]*len(theta)
  for i in range(len(X)):
    logit = inner(X[i], theta)
    for k in range(len(theta)):
      dl[k] += X[i][k] * (1 - sigmoid(logit))
      if not y[i]:
        dl[k] -= X[i][k]
        dl[k] -= (X[i][k]*sigmoid(logit))*(ratio - 1)
  for k in range(len(theta)):
    dl[k] -= lam*2*theta[k]
  return numpy.array([-x for x in dl])

temp1 = (sum(y_training_set))
temp2 = (len(y_training_set) - sum(y_training_set))
print ("Ratio: " + str(temp1/temp2))
ratio = temp1/temp2

# define lambda
lam = 1.0

# train on training set
theta = train(lam, X_training_set, y_training_set)
print ("Training done!")

# evaluate on training set
acc = error_statistics(theta, X_training_set, y_training_set)
print ("Balanced Error Rate on training set: " + str(acc[4]))

# evaluate on validation set
acc = error_statistics(theta, X_validation_set, y_validation_set)
print ("Balanced Error Rate on validation set: " + str(acc[4]))

# evaluate on test set
acc = error_statistics(theta, X_test_set, y_test_set)
print ("Balanced Error Rate on test set: " + str(acc[4]))

Ratio: 1.2773981962284777
Training done!
Balanced Error Rate on training set: 0.4342908816345251
Balanced Error Rate on validation set: 0.4108055686954606
Balanced Error Rate on test set: 0.4434130314950322


In [10]:
# Problem 5

def pipeline(training_set_X, training_set_y, validation_set_X, validation_set_y, test_set_X, test_set_y, lam):
    print ("Lambda: " + str(lam))
    
    theta = train(lam, training_set_X, training_set_y)
    print ("Training done!")
    
    # evaluate on training set
    acc = error_statistics(theta, training_set_X, training_set_y)
    print ("Balanced Error Rate on training set: " + str(acc[4]))

    # evaluate on validation set
    acc = error_statistics(theta, validation_set_X, validation_set_y)
    print ("Balanced Error Rate on validation set: " + str(acc[4]))

    # evaluate on test set
    acc = error_statistics(theta, test_set_X, test_set_y)
    print ("Balanced Error Rate on test set: " + str(acc[4]))
    
lambda_values = [0, 0.01, 0.1, 1, 100]

for i in range (0,len(lambda_values)):
    pipeline(X_training_set, y_training_set, X_validation_set, y_validation_set, X_test_set, y_test_set, lambda_values[i])

Lambda: 0
Training done!
Balanced Error Rate on training set: 0.4343295317096092
Balanced Error Rate on validation set: 0.4110884452355663
Balanced Error Rate on test set: 0.44343425980765727
Lambda: 0.01
Training done!
Balanced Error Rate on training set: 0.4342908816345251
Balanced Error Rate on validation set: 0.4108055686954606
Balanced Error Rate on test set: 0.4434130314950322
Lambda: 0.1
Training done!
Balanced Error Rate on training set: 0.4342908816345251
Balanced Error Rate on validation set: 0.4108055686954606
Balanced Error Rate on test set: 0.4434130314950322
Lambda: 1
Training done!
Balanced Error Rate on training set: 0.4342908816345251
Balanced Error Rate on validation set: 0.4108055686954606
Balanced Error Rate on test set: 0.4434130314950322
Lambda: 100
Training done!
Balanced Error Rate on training set: 0.4342967434651259
Balanced Error Rate on validation set: 0.4106169843353902
Balanced Error Rate on test set: 0.44348478319170515
